In [1]:
%load_ext autoreload
%autoreload 2

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import kuaizi as kz
from astropy.table import Table
import astropy.units as u

from kuaizi.display import display_single
from astropy.io import fits
import os
import sys
from shutil import copyfile

In [4]:
kz.utils.set_env(project='HSC', name='LSBG', data_dir='/scratch/gpfs/jiaxuanl/Data/')

'/scratch/gpfs/jiaxuanl/Data/HSC/LSBG'

In [5]:
cat = Table.read('./Catalog/nsa_test_sample_210927.fits')

### Generate RGB images

In [23]:
def _display_cutout_rgb(images, ax=None):
    import scarlet
    from scarlet.display import AsinhMapping

    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))
        
    # Display
    norm = AsinhMapping(minimum=-0.2, stretch=1, Q=1)
    img_rgb = scarlet.display.img_to_rgb(images, norm=norm)
    plt.imshow(img_rgb, origin='lower')
    ax.axis('off')
    
    if ax is None:
        return fig
    return ax

In [39]:
CUTOUT_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/'
FIGURE_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/'
WEBPAGE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/figure/'

In [40]:
# Check file exsistence
for filt in 'grizy':
    for obj in cat:
        if not os.path.isfile(CUTOUT_DIR + f"nsa_{obj['viz-id']}_{filt}.fits"):
            print(f"NSA/nsa_{obj['viz-id']}_{filt}.fits not found!")

In [28]:
# Display each cutout
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    
    images = np.asarray([fits.open(f'{CUTOUT_DIR}/nsa_{ind}_{filt}.fits')[1].data for filt in 'grizy'])
    fig, ax = plt.subplots(figsize=(5, 5))
    ax = _display_cutout_rgb(images, ax=ax)
    plt.savefig(FIGURE_DIR + f'nsa_{ind}_cutout.png', dpi=70, bbox_inches='tight')
    plt.close()
    if i % 10 == 0:
        print(f'Progress: {i} / {len(cat)}')

Progress: 0 / 50
Progress: 10 / 50
Progress: 20 / 50
Progress: 30 / 50
Progress: 40 / 50


In [41]:
# Copy figures in FIGURE_DIR to WEBPAGE_DIR
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    copyfile(FIGURE_DIR + f'nsa_{ind}_cutout.png', WEBPAGE_DIR + f'nsa_{ind}_cutout.png')

### Generate webpage for cutouts. WE use `diezi/webpage/gen_web_cutout.py`

In [46]:
sys.path.append('/home/jiaxuanl/Research/Packages/kuaizi/diezi/webpage/')
from gen_web_cutout import webpage_cutout

In [77]:
#FOR NSA SAMPLE
FIGURE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/figure'
WEBPAGE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/'
SCARLET_DIR = '/tigress/jiaxuanl/public_html/candy/gpfs_scarlet_zoomin/'
sample_name = 'NSA'

In [78]:
webpage_cutout(FIGURE_DIR, WEBPAGE_DIR, sample_name, SCARLET_DIR=SCARLET_DIR, 
               col_num=6, row_num=4)

You have 50 galaxies to be displayed
Total pages: 3


In [79]:
print('The webpages are in', WEBPAGE_DIR)

The webpages are in /tigress/jiaxuanl/public_html/NSA/cutout_RGB/


### Search box still doesn't work. Need to work on the javascript file